## Exploring changing crops in Georgia

Interestingly, one of the major crops which are grown in Georgia are
blueberries (despite it being called the peach state); in particular,
it is the number one state in the nation for producing peanuts, chickens,
pecans and blueberries, with chickens being the number one commodity by value,
and blueberries the tenth (with just over 300 million dollars worth of
blueberries being produced). That said, there are a few things which threaten
the blueberry crop:

- Climate change:
- Crop diseases:

Other things I've learnt:
- Chickens are called "broilers" in the context of agricultural
products. Don't ask me why.
- Top five blueberry producing counties (by value): Bacon, Clinch, 
Appling, Ware, Coffee
- Top ten commodities by value: Broilers, Cotton, Eggs, Timber, Peanuts,
Beef, Greenhouse, Dairy, Pecans, Blueberries

(Things to update in the code quickly: include layers for these crops,
along with crops people are suggesting/investigating as replacements)

### Sources

- [Information about Georgia's commodities](https://www.gfb.org/education-and-outreach/about-ga-agriculture.cms)
- [Source for top producing counties](https://extension.uga.edu/topic-areas/fruit-vegetable-ornamentals-production/blueberries.html)
- [Bacon County Agriculture & Natural Resources](https://extension.uga.edu/county-offices/bacon/agriculture-and-natural-resources.html)
- [Cropland data on GEE](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL)

## Visualizing Georgia overall

In [8]:
import ee
import geemap
import pandas as pd
import geopandas as gpd

ee.Initialize()

def pd_shp_to_ee_poly(shp):
    """Converts Polygon from GeoPandas to a ee.Geometry.Polygon
    object suitable for use within Google Earth Engine."""
    xs, ys = shp.exterior.coords.xy
    shp_list = [[x, y] for x, y in zip(xs, ys)]
    roi = ee.Geometry.Polygon(shp_list, None, False)
    return roi

def maskCrops(image, vals_to_keep):
    """Masks values of the image to only include those
    within vals_to_keep."""
    masks = []
    finalMask = ee.Image(0)

    for val in vals_to_keep:
        masks.append(image.eq(val))
    
    for mask in masks:
        finalMask = finalMask.Or(mask) 
    
    return image.updateMask(finalMask)

In [9]:
ga_shp = gpd.read_file("ga-boundary/ga.shp").geometry[0]
ga_roi = pd_shp_to_ee_poly(ga_shp).simplify(maxError = 1)
print(ga_shp.centroid)

POINT (-83.42713697518201 32.63861886551485)


In [3]:
collection = (
    ee.ImageCollection('USDA/NASS/CDL')
    .filterDate('2008-01-01', '2021-01-01')
    .select('cropland')
    .filterBounds(ga_roi)
    .map(lambda image: image.clip(ga_roi))
)

In [4]:
class_vals = collection.first().get("cropland_class_values").getInfo()
class_labels = collection.first().get("cropland_class_names").getInfo()
class_palette = collection.first().get("cropland_class_palette").getInfo()

crop_classes = pd.DataFrame(
    {
        'layer_vals': class_vals,
        'labels': class_labels,
        'palette': class_palette
    }
)

crop_classes.head()

,layer_vals,labels,palette
0,0,Background,000000
1,1,Corn,ffd400
2,2,Cotton,ff2626
3,3,Rice,00a8e3
4,4,Sorghum,ff9e0a


In [5]:
crop_types = [10, 67, 242, 2]
legend_keys = list(crop_classes[crop_classes.layer_vals.isin(crop_types)].labels)
legend_colors = list(crop_classes[crop_classes.layer_vals.isin(crop_types)].palette)
print(legend_keys, legend_colors)

['Cotton', 'Peanuts', 'Peaches', 'Blueberries'] ['ff2626', '70a600', 'ff8fab', '000099']


In [6]:
filtered_collection = collection.map(lambda image: maskCrops(image, crop_types))

Map = geemap.Map()
Map.setCenter(-83.42713, 32.63861, 8)
Map.addLayer(ga_roi, {'opacity': 0.2}, 'Georgia')
Map.addLayer(
    filtered_collection.first(), {}, "First image"
)
Map.add_time_slider(
    filtered_collection, 
    {}, 
    region = ga_roi,
    time_interval = 1
)
Map.add_legend(
    legend_keys = legend_keys, 
    legend_colors = legend_colors, 
    position = 'topright'
)
Map

Map(center=[32.63861, -83.42713], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [13]:
vals = filtered_collection.first().reduceRegion(
    ee.Reducer.frequencyHistogram(), bestEffort = True
)

crop_vals = vals.getInfo()
crop_vals

{'cropland': {'10': 169151.81176470587,
  '2': 238773.26666666666,
  '242': 4948,
  '67': 1759}}

## Georgia landcover

In [19]:
collection = (
    ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
    .select('landcover')
    .filterBounds(ga_roi)
    .map(lambda image: image.clip(ga_roi))
)


def maskCrops(image, vals_to_keep):
    """Masks values of the image to only include those
    within vals_to_keep."""
    masks = []
    finalMask = ee.Image(0)

    for val in vals_to_keep:
        masks.append(image.eq(val))
    
    for mask in masks:
        finalMask = finalMask.Or(mask) 
    
    return image.updateMask(finalMask)


class_vals = collection.first().get("landcover_class_values").getInfo()
class_labels = collection.first().get("landcover_class_names").getInfo()
class_palette = collection.first().get("landcover_class_palette").getInfo()

class_df = pd.DataFrame(
    {
        'layer_vals': class_vals,
        'labels': class_labels,
        'palette': class_palette
    }
)

In [20]:
collection.size().getInfo()

8

In [ ]:
class_vals = collection.first().get("landcover_class_values").getInfo()
class_labels = collection.first().get("landcover_class_names").getInfo()
class_palette = collection.first().get("landcover_class_palette").getInfo()

class_df = pd.DataFrame(
    {
        'layer_vals': class_vals,
        'labels': class_labels,
        'palette': class_palette
    }
)

values_to_keep = [21, 22, 23, 24, 31, 51, 52]

legend_keys = list(class_df[class_df.layer_vals.isin(values_to_keep)].labels)
legend_colors = list(class_df[class_df.layer_vals.isin(values_to_keep)].palette)

filtered_collection = collection.map(lambda image: maskCrops(image, values_to_keep))

Map = geemap.Map()
Map.setCenter(-83.42713, 32.63861, 8)
Map.addLayer(ga_roi, {'opacity': 0.2}, 'Georgia')
Map.addLayer(
    filtered_collection.first(), {}, "First image"
)
Map.add_time_slider(
    filtered_collection, 
    {}, 
    region = ga_roi,
    time_interval = 1
)
Map.add_legend(
    legend_keys = legend_keys, 
    legend_colors = legend_colors, 
    position = 'topright'
)
Map

## Focusing on some of the individual counties

In [34]:
def get_county_roi(county_name):
    """Returns a ee.Geometry.Polygon object representing
    a particular county within Georgia, along with the
    centroid of that object."""
    ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
    county_shp = ga_counties[ga_counties["NAME10"] == county_name].geometry.values[0]
    xs, ys = county_shp.centroid.coords.xy
    county_roi = pd_shp_to_ee_poly(county_shp).simplify(maxError = 1)

    return county_roi, (xs[0], ys[0])

def get_labels(collection):
    """Returns a data frame containing the band values/class/corresponding
    palette color."""
    class_vals = collection.first().get("cropland_class_values").getInfo()
    class_labels = collection.first().get("cropland_class_names").getInfo()
    class_palette = collection.first().get("cropland_class_palette").getInfo()

    crop_classes = pd.DataFrame({
        'layer_vals': class_vals,
        'labels': class_labels,
        'palette': class_palette
    })

    return crop_classes

def get_legend_keys_values(class_df, crops_to_keep):
    legend_keys = list(class_df[class_df.layer_vals.isin(crops_to_keep)].labels)
    legend_colors = list(class_df[class_df.layer_vals.isin(crops_to_keep)].palette)
    return legend_keys, legend_colors

def produce_map(collection, roi, centroid, crops_to_keep, county_name):
    """Returns a map object for a given collection of cropland."""
    filtered_collection = collection.map(
        lambda image: maskCrops(image, crops_to_keep)
    )

    class_df = get_labels(collection)
    legend_keys, legend_colors = get_legend_keys_values(class_df, crops_to_keep)
    
    Map = geemap.Map()
    Map.setCenter(centroid[0], centroid[1], 11)
    Map.addLayer(
        roi, {'opacity': 0.2}, county_name + ' County'
    )
    Map.addLayer(
        filtered_collection.first(), {}, "First image"
    )
    Map.add_time_slider(
        filtered_collection, 
        {}, 
        region = roi,
        time_interval = 2
    )
    Map.add_legend(
        legend_keys = legend_keys, 
        legend_colors = legend_colors, 
        position = 'topright'
    )
    return Map

In [36]:
county_name = 'Bacon'
crops_to_keep = [10, 67, 242, 2]
roi, centroid = get_county_roi(county_name)
collection = (
    ee.ImageCollection('USDA/NASS/CDL') 
    .filterDate('2008-01-01', '2021-01-01') 
    .select('cropland') 
    .filterBounds(roi)
    .map(lambda image: image.clip(roi))
)

bacon_map = produce_map(collection, roi, centroid, crops_to_keep, county_name)
bacon_map

Map(center=[31.553680822688413, -82.4526886972915], controls=(WidgetControl(options=['position', 'transparent_…

In [37]:
county_name = 'Coffee'
crops_to_keep = [10, 67, 242, 2]
roi, centroid = get_county_roi(county_name)
collection = (
    ee.ImageCollection('USDA/NASS/CDL') 
    .filterDate('2008-01-01', '2021-01-01') 
    .select('cropland') 
    .filterBounds(roi)
    .map(lambda image: image.clip(roi))
)

coffee_map = produce_map(collection, roi, centroid, crops_to_keep, county_name)
coffee_map

Map(center=[31.54927769494608, -82.84920456905367], controls=(WidgetControl(options=['position', 'transparent_…

To explore further:
- extracting values of crops of interest on a county level basis
for each year, understand trends in the areas where people are
growing particular types of crops
- see if there is a growth/shrinkage in the types of crops farmers
are growing over time

In [33]:
vals = collection.first().reduceRegion(
    ee.Reducer.frequencyHistogram()
)

crop_vals = vals.getInfo()

In [54]:
crop_vals_df = (
    pd.DataFrame.from_dict(crop_vals)
    .rename_axis(index = 'values')
    .reset_index()
).astype({'values': 'int64', 'cropland': 'float64'})

crop_vals_df.head()
crop_vals_df.dtypes

values        int64
cropland    float64
dtype: object

In [55]:
crop_vals_df.merge(
    crop_classes, on = "values"
)

,values,cropland,labels,palette
0,1,18490.788235,Corn,ffd400
1,10,35415.050980,Peanuts,70a600
2,11,1643.352941,Tobacco,00b04a
3,111,2403.003922,Open Water,4a70a3
4,12,4.000000,Sweet Corn,dea60a
5,121,36042.572549,Developed/Open Space,999999
6,122,14268.454902,Developed/Low Intensity,999999
7,123,2557.447059,Developed/Med Intensity,999999
8,124,678.000000,Developed/High Intensity,999999
9,131,44.725490,Barren,ccc0a3
